In [1]:
import pandas as pd

df = pd.read_csv("preprocessed_dataset.csv",nrows=1000000)
print(df.head())

   Unnamed: 0  type       rcvTime     pos_0     pos_1  pos_noise_0  \
0           0     3  27371.216284  0.671505  0.520136     0.000274   
1           1     3  52060.561117  0.692568  0.551755     0.000321   
2           2     3  28156.319142  0.634308  0.550787     0.000212   
3           3     3  28671.375689  0.592830  0.512949     0.000353   
4           4     2  53612.000000  0.590575  0.476267     0.000238   

   pos_noise_1     spd_0     spd_1  spd_noise_0  ...     acl_0     acl_1  \
0     0.000284  0.158960  0.365369     0.390111  ...  0.473667  0.480255   
1     0.000319  0.391546  0.404605     0.474784  ...  0.447554  0.473846   
2     0.000220  0.409171  0.429974     0.518330  ...  0.472780  0.336080   
3     0.000361  0.539499  0.472207     0.641699  ...  0.486963  0.354515   
4     0.000242  0.502468  0.396957     0.521358  ...  0.452836  0.490056   

   acl_noise_0  acl_noise_1     hed_0     hed_1  hed_noise_0  hed_noise_1  \
0     0.006141     0.008928  0.014335  0.3811

In [2]:
import torch
from sklearn.model_selection import train_test_split

# Separate features and labels
X = df.drop(columns=['attack'])  # Features
y = df['attack']  # Target (0 = Benign, 1 = Attack)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

print("Data prepared for Deep Learning!")

Data prepared for Deep Learning!


In [3]:
import torch.nn as nn
import torch.optim as optim

# Define MLP model
class DDoSClassifier(nn.Module):
    def __init__(self, input_size):
        super(DDoSClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(64, 2)  # Binary Classification (0 = Benign, 1 = Attack)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu2(self.fc2(x))
        x = self.softmax(self.fc3(x))
        return x

# Initialize model
input_size = X_train.shape[1]  # Number of features
model = DDoSClassifier(input_size)

print(model)


DDoSClassifier(
  (fc1): Linear(in_features=20, out_features=128, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=64, out_features=2, bias=True)
  (softmax): LogSoftmax(dim=1)
)


In [4]:
# Loss function & optimizer
criterion = nn.CrossEntropyLoss()  # Since we have two classes (0,1)
optimizer = optim.Adam(model.parameters(), lr=0.001)

print("Loss function & optimizer defined!")


Loss function & optimizer defined!


In [5]:
# Training loop
epochs = 30  # Start with 30 epochs, increase if needed
batch_size = 1024
train_loader = torch.utils.data.DataLoader(list(zip(X_train_tensor, y_train_tensor)), batch_size=batch_size, shuffle=True)

for epoch in range(epochs):
    model.train()
    total_loss = 0

    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}")

print("Model training complete!")


Epoch 1/30, Loss: 307545.0334
Epoch 2/30, Loss: 100699.8412
Epoch 3/30, Loss: 28406.5014
Epoch 4/30, Loss: 15393.4194
Epoch 5/30, Loss: 4713.5533
Epoch 6/30, Loss: 541.3847
Epoch 7/30, Loss: 539.7202
Epoch 8/30, Loss: 539.5699
Epoch 9/30, Loss: 539.1819
Epoch 10/30, Loss: 538.8022
Epoch 11/30, Loss: 538.5107
Epoch 12/30, Loss: 538.5031
Epoch 13/30, Loss: 538.3414
Epoch 14/30, Loss: 538.3692
Epoch 15/30, Loss: 538.4352
Epoch 16/30, Loss: 538.5792
Epoch 17/30, Loss: 538.5846
Epoch 18/30, Loss: 538.5899
Epoch 19/30, Loss: 538.5987
Epoch 20/30, Loss: 538.5958
Epoch 21/30, Loss: 538.5904
Epoch 22/30, Loss: 538.5912
Epoch 23/30, Loss: 538.5857
Epoch 24/30, Loss: 538.5996
Epoch 25/30, Loss: 538.5839
Epoch 26/30, Loss: 538.5948
Epoch 27/30, Loss: 538.5924
Epoch 28/30, Loss: 538.5937
Epoch 29/30, Loss: 538.5885
Epoch 30/30, Loss: 538.5974
Model training complete!


In [7]:
# Evaluate on test data
model.eval()
with torch.no_grad():
    y_pred_test = model(X_test_tensor)
    y_pred_labels = torch.argmax(y_pred_test, axis=1)

# Calculate accuracy
accuracy = (y_pred_labels == y_test_tensor).float().mean().item()
print(f"Test Accuracy: {accuracy:.4f}")


Test Accuracy: 0.5470
